In [71]:
import pickle
import json
!pip install redshift_connector
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import re
import os
!pip install unidecode
from unidecode import unidecode
from collections import Counter
from math import ceil
!pip install langdetect
from langdetect import detect
from random import sample

## Support Files

Throughout the modeling process, some of the model artifacts needed to be updated and so this notebook was used to quickly update those files.

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Cambiar path**

In [73]:
import os
base_path = '/content/drive/MyDrive/openalex-institution-parsing/'
# location where current files are located
curr_model_artifacts_location = os.path.join(base_path, 'Crudos/institution_tagger_v2_artifacts/')


In [74]:


# Load the needed files
with open(f"{curr_model_artifacts_location}departments_list.pkl", "rb") as f:
    departments_list = pickle.load(f)
print("Loaded list of departments")

with open(f"{curr_model_artifacts_location}countries_list_flat.pkl", "rb") as f:
    countries_list_flat = pickle.load(f)
print("Loaded flat list of countries")

with open(f"{curr_model_artifacts_location}countries.json", "r") as f:
    countries_dict = json.load(f)
print("Loaded countries dictionary")

with open(f"{curr_model_artifacts_location}city_country_list.pkl", "rb") as f:
    city_country_list = pickle.load(f)
print("Loaded strings of city/country combinations")

Loaded list of departments
Loaded flat list of countries
Loaded countries dictionary
Loaded strings of city/country combinations


### Looking at ROR

In [75]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [76]:
def get_final_region(ror_state, ror_region):
    if isinstance(ror_state, str):
        return ror_state
    elif isinstance(ror_region, str):
        return ror_region
    else:
        return None

In [77]:
def check_for_backwards_univ(curr_names):
    names = curr_names.copy()
    for one_name in curr_names:
        split_name = one_name.split(" ")
        if len(split_name) == 3:
            if (split_name[0] == 'University') & (split_name[1] == 'of'):
                names.append(f"{split_name[2]} University")
        elif len(split_name) == 2:
            if (split_name[1] == 'University'):
                names.append(f"University of {split_name[0]}")
        else:
            pass
    return names

In [78]:
def add_names_to_list(all_names):
    names = all_names.copy()
    if "Harvard University" in names:
        names.append("Harvard Medical School")
    elif "University of Oxford" in names:
        names.append("Oxford University")
    else:
        pass

    return names

In [79]:
def get_exact_names(name, aliases, acronyms, labels):
    all_names = [name] + aliases + acronyms + [i['label'] for i in labels]
    all_names = add_names_to_list(all_names)
    all_names = [x for x in all_names if ~x.startswith('Department of')]
    all_names_clean = [string_match_clean(x) for x in all_names]
    return [x for x in all_names_clean if len(x) > 4]

In [80]:
import pickle
import json
!pip install redshift_connector
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import re
import os
!pip install unidecode
from unidecode import unidecode
from collections import Counter
from math import ceil
!pip install langdetect
from langdetect import detect
from random import sample

!pip install pyalex
import pyalex

In [81]:
def string_match_clean(text):
    #replace "&" with "and"
    if "r&d" not in text.lower():
        text = text.replace(" & ", " and ")

    # take country out
    if text.strip().endswith(")"):
        for country in countries_list_flat:
            if text.strip().endswith(f"({country})"):
                text = text.replace(f"({country})", "")

    # use unidecode
    text = unidecode(text.strip())

    # replacing common abbreviations
    text = text.replace("Univ.", "University")
    text = text.replace("Lab.", "Laboratory")

    # take out spaces, commas, dashes, periods, etcs
    text = re.sub("[^0-9a-zA-Z]", "", text)

    return text

In [82]:
def list_of_all_names(oa_name, ror_names, extra_names, use_extra_names=False):
    banned_names = ['UniversityHospital','Coastal','Brunswick','Continental']
    if isinstance(ror_names, list):
        pass
    else:
        ror_names = []

    if isinstance(oa_name, str):
        oa_string = [string_match_clean(oa_name)]
    else:
        oa_string = []

    if use_extra_names:
        if isinstance(extra_names, list):
            pass
        else:
            extra_names = []
    else:
        extra_names = []

    return [x for x in list(set(oa_string+ror_names+extra_names)) if
            ((len(x) > 4) & (x not in banned_names))]

In [83]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id','display_name'
#institutions_df = pd.read_parquet("/content/drive/MyDrive/openalex-institution-parsing/Archivos/OA_static_institutions_single_file.parquet")

### ALTERNATIVA DE OA_static_institutions_single_file.parquet bajando desde Python

from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
pyalex.config.email = "mfartola.mincyt@gmail.com"

from pyalex import config
config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

In [84]:
import json,urllib.request
data = urllib.request.urlopen("https://api.openalex.org/institutions?filter=country_code:AR&per-page=200&page=1").read()
data1 = urllib.request.urlopen("https://api.openalex.org/institutions?filter=country_code:AR&per-page=200&page=2").read()

output = json.loads(data)
output1 = json.loads(data1)

argentina1=pd.DataFrame(output['results'])
argentina2=pd.DataFrame(output1['results'])
argentina=pd.concat([argentina1,argentina2])
argentina=pd.DataFrame(argentina)
argentina.head()
argentina.shape
institutions_df=argentina
institutions_df.columns

Index(['id', 'ror', 'display_name', 'country_code', 'type', 'type_id',
       'lineage', 'homepage_url', 'image_url', 'image_thumbnail_url',
       'display_name_acronyms', 'display_name_alternatives', 'repositories',
       'works_count', 'cited_by_count', 'summary_stats', 'ids', 'geo',
       'international', 'associated_institutions', 'counts_by_year', 'roles',
       'topics', 'topic_share', 'x_concepts', 'is_super_system',
       'works_api_url', 'updated_date', 'created_date'],
      dtype='object')

In [85]:
institutions_df['affiliation_id'] = institutions_df['id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id']=institutions_df['affiliation_id'].apply(lambda x: x.split("I")[-1])
#institutions_df['affiliation_id']=institutions_df['affiliation_id'].astype(int)  ### paso los ids como enteros

institutions_df['ror_id'] = institutions_df['ror'].apply(lambda x: x.split("/")[-1])

#institutions_df['display_name'] = institutions_df['display_name_alternatives'] ###ojo, lo comenté!

institutions_df.head(1)

id                        ror  \
0  https://openalex.org/I151201029  https://ror.org/03cqe8w59   

                                                 display_name country_code  \
0  Consejo Nacional de Investigaciones Científicas y Técnicas           AR   

         type                                            type_id  \
0  government  https://openalex.org/institution-types/government   

                                                                                                 lineage  \
0  [https://openalex.org/I151201029, https://openalex.org/I4210123736, https://openalex.org/I4387155568]   

                homepage_url  \
0  http://www.conicet.gov.ar   

                                                                                                 image_url  \
0  https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Conicet%20Logo%20con%20letras.png   

                                                                                                 image_thumbnail_url  \
0  https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Conicet%20Logo%20con%20letras.png&width=300   

  display_name_acronyms                             display_name_alternatives  \
0             [CONICET]  [National Scientific and Technical Research Council]   

  repositories  works_count  cited_by_count  \
0           []       149878         2537832   

                                                                    summary_stats  \
0  {'2yr_mean_citedness': 2.5566296479474846, 'h_index': 366, 'i10_index': 55063}   

                                                                                                                                                                                                                                                                                                 ids  \
0  {'openalex': 'https://openalex.org/I151201029', 'ror': 'https://ror.org/03cqe8w59', 'mag': '151201029', 'grid': 'grid.423606.5', 'wikipedia': 'https://en.wikipedia.org/wiki/National%20Scientific%20and%20Technical%20Research%20Council', 'wikidata': 'https://www.wikidata.org/wiki/Q6978293'}   

                                                                                                                                                                    geo  \
0  {'city': 'Buenos Aires', 'geonames_city_id': '3435910', 'region': None, 'country_code': 'AR', 'country': 'Argentina', 'latitude': -34.61315, 'longitude': -58.37723}   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    international  \
0  {'display_name': {'ast': 'Conseyu Nacional d'Investigaciones Científiques y Téuniques', 'ca': 'Consell Nacional d'Investigacions Científiques i Tècniques', 'cy': 'Y Cyngor Ymchwil Cenedlaethol Gwyddonol a Thechnegol', 'en': 'National Scientific and Technical Research Council', 'eo': 'Nacia Konsilio por Scienca kaj Teknika Esploro de Argentino', 'es': 'Consejo Nacional de Investigaciones Científicas y Técnicas', 'eu': 'Consejo Nacional de Investigaciones Científicas y Técnicas (Argentina)', 'fr': 'Conseil national de la recherche scientifique et technique', 'he': 'מועצת המ

In [86]:
institutions_df.columns

Index(['id', 'ror', 'display_name', 'country_code', 'type', 'type_id',
       'lineage', 'homepage_url', 'image_url', 'image_thumbnail_url',
       'display_name_acronyms', 'display_name_alternatives', 'repositories',
       'works_count', 'cited_by_count', 'summary_stats', 'ids', 'geo',
       'international', 'associated_institutions', 'counts_by_year', 'roles',
       'topics', 'topic_share', 'x_concepts', 'is_super_system',
       'works_api_url', 'updated_date', 'created_date', 'affiliation_id',
       'ror_id'],
      dtype='object')

In [87]:
institutions_df=institutions_df[['affiliation_id','display_name','ror_id']]
institutions_df.head(2)

,affiliation_id,display_name,ror_id
0,151201029,Consejo Nacional de Investigaciones Científicas y Técnicas,03cqe8w59
1,24354313,University of Buenos Aires,0081fs513


In [88]:
##Carga de ror

file_path = os.path.join(base_path, 'Crudos/v1.53-2024-10-14-ror-data.json')

ror = pd.read_json(file_path)
ror.head(2)
ror.columns

Index(['id', 'name', 'types', 'links', 'aliases', 'acronyms', 'status',
       'wikipedia_url', 'labels', 'email_address', 'ip_addresses',
       'established', 'country', 'relationships', 'addresses', 'external_ids'],
      dtype='object')

In [89]:
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['ror_id']=ror['ror_id'].apply(lambda x: x.split("I")[-1])

#ror['types'] = ror['types'].apply(lambda x: x[0])
ror['types'] = ror['types'].apply(lambda x: x[0] if len(x)>0 else "")
##
#ror[ror['types'].isnull()]

In [90]:
ror['country_code'] = ror['country'].apply(lambda x: x['country_code'])
ror['country_name'] = ror['country'].apply(lambda x: x['country_name'])
ror['city'] = ror['address'].apply(lambda x: x['city'])
ror['state'] = ror['address'].apply(lambda x: x['state'])
ror['region'] = ror['address'].apply(get_geoname_admin)

In [91]:
ror.columns

Index(['id', 'name', 'types', 'links', 'aliases', 'acronyms', 'status',
       'wikipedia_url', 'labels', 'email_address', 'ip_addresses',
       'established', 'country', 'relationships', 'addresses', 'external_ids',
       'address', 'ror_id', 'country_code', 'country_name', 'city', 'state',
       'region'],
      dtype='object')

In [92]:
ror_to_join = ror[['ror_id','name','status','types','aliases','acronyms','labels','city',
                   'state','region','country_name']].copy()

ror_to_join.columns = ['ror_id','name','status','types','aliases','acronyms','labels','city',
                       'temp_state','temp_region','country']

In [93]:
ror_to_join['state'] = ror_to_join.apply(lambda x: get_final_region(x.temp_state, x.temp_region), axis=1)
ror_to_join.head(1)

,ror_id,name,status,types,aliases,acronyms,labels,city,temp_state,temp_region,country,state
0,04ttjf776,RMIT University,active,Education,[Royal Melbourne Institute of Technology University],[RMIT],[],Melbourne,Victoria,Victoria,Australia,Victoria


In [94]:
institutions_df.head(1)

,affiliation_id,display_name,ror_id
0,151201029,Consejo Nacional de Investigaciones Científicas y Técnicas,03cqe8w59


In [95]:
inst_ror = ror_to_join.merge(institutions_df[['ror_id','affiliation_id','display_name']],how='inner', on='ror_id')
inst_ror.shape

(331, 14)

In [96]:
inst_ror.head(n=3)

,ror_id,name,status,types,aliases,acronyms,labels,city,temp_state,temp_region,country,state,affiliation_id,display_name
0,00pt8r998,National University of the Littoral,active,Education,[],[UNL],"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",Santa Fe,None,Santa Fe,Argentina,Santa Fe,3121987637,National University of the Littoral
1,00htwgm11,National University of Salta,active,Education,[],[UNSa],"[{'label': 'Universidad Nacional de Salta', 'iso639': 'es'}]",Salta,None,Salta,Argentina,Salta,10108838,National University of Salta
2,01yjy8p80,National University of Luján,active,Education,[],[],"[{'label': 'Universidad Nacional de Luján', 'iso639': 'es'}]",Luján,None,Buenos Aires,Argentina,Buenos Aires,168807105,National University of Luján


#### Getting file of multi-institution names

In [97]:
multi_inst_names_df = ror_to_join[['ror_id','name']].merge(institutions_df[['ror_id','affiliation_id']],
                                                        how='left', on='ror_id') \
['name'].value_counts().reset_index()

In [98]:
multi_inst_names_df.head(10)

,name,count
0,Ministry of Health,53
1,Ministry of Education,23
2,Argosy University,19
3,Ministry of Justice,16
4,Ministry of Foreign Affairs,16
5,Ministry of Culture,13
6,Government Medical College,13
7,St. Luke's Hospital,12
8,International Institute of Tropical Agriculture,11
9,Ministry of Finance,11


In [99]:
multi_inst_names = multi_inst_names_df[multi_inst_names_df['count']>1]
multi_inst_names
multi_inst_names=multi_inst_names['name'].tolist()
#multi_inst_names

In [100]:
## paso a entero a los ids antes de pasarlo a lista
multi_inst_names_ids = inst_ror[inst_ror['name'].isin(multi_inst_names)]['affiliation_id'].astype(int).tolist()

In [101]:
with open(f"{curr_model_artifacts_location}multi_inst_names_ids1.pkl", "wb") as f:
    pickle.dump(multi_inst_names_ids, f)

In [102]:
multi_inst_names_ids

[4210139084,
 4210110275,
 4210098121,
 1333839041,
 4210158077,
 4210123859,
 4210123736]

### Getting Mapping of Inactive Institutions

There are institutions in ROR that are listed as "Withdrawn" or "Inactive". There was some thought to use the old data associated with these ROR IDs and apply them to successors but for this model, we decided to hold off on doing this because we were unsure if there would be a benefit to doing so. Therefore, the code is provided but this data was not used in building the model.

In [103]:
def get_successors_from_relationships(relationships):
    successors = []
    parents = []
    for relationship in relationships:
        if relationship['type'] == 'Successor':
            successors.append(relationship['id'].split("/")[-1])
        elif relationship['type'] == 'Parent':
            parents.append(relationship['id'].split("/")[-1])
        else:
            pass
    return [successors, parents]

In [104]:
def get_extra_names(ror_id):
    if ror_id in successor_dict.keys():
        extra_names = []
        for old_id in successor_dict[ror_id]['ror_id']:
            extra_names += old_name_data[old_id]['successor_names']

        extra_names = list(set(extra_names))
    else:
        extra_names = []

    return extra_names

In [105]:
withdrawn_or_inactive_df = ror[ror['status'].isin(['withdrawn','inactive'])].copy()
withdrawn_or_inactive_df.shape

(1973, 23)

In [106]:
withdrawn_or_inactive_df['successors_parents'] = withdrawn_or_inactive_df['relationships'] \
    .apply(get_successors_from_relationships)

In [107]:
withdrawn_or_inactive_df['successors'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[0])
withdrawn_or_inactive_df['parents'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[1])

In [108]:
withdrawn_or_inactive_df['successor_len'] = withdrawn_or_inactive_df['successors'].apply(len)

In [109]:
to_add_to_successors = withdrawn_or_inactive_df[withdrawn_or_inactive_df['successor_len']==1].copy()
to_add_to_successors['successor'] = to_add_to_successors['successors'].apply(lambda x: x[0])
to_add_to_successors.shape

(1580, 28)

In [110]:
to_add_to_successors['successor_names'] = to_add_to_successors.apply(lambda x: get_exact_names(x['name'],
                                                                                               x.aliases,
                                                                                               x.acronyms,
                                                                                               x.labels),
                                                                     axis=1)

In [111]:
old_name_data = to_add_to_successors.set_index('ror_id')[['successor_names']].to_dict(orient='index')

In [112]:
successor_dict = to_add_to_successors.groupby('successor')['ror_id'].apply(list).reset_index()\
    .set_index('successor').to_dict(orient='index')

### Getting ROR String Matching File and Affiliation Dictionary

In [113]:
inst_ror['extra_names'] = inst_ror['ror_id'].apply(get_extra_names)
inst_ror['extra_names']


,extra_names
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
326,[]
327,[]
328,[]
329,[]


In [114]:
import numpy as np

In [115]:
inst_ror.head()

,ror_id,name,status,types,aliases,acronyms,labels,city,temp_state,temp_region,country,state,affiliation_id,display_name,extra_names
0,00pt8r998,National University of the Littoral,active,Education,[],[UNL],"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",Santa Fe,None,Santa Fe,Argentina,Santa Fe,3121987637,National University of the Littoral,[]
1,00htwgm11,National University of Salta,active,Education,[],[UNSa],"[{'label': 'Universidad Nacional de Salta', 'iso639': 'es'}]",Salta,None,Salta,Argentina,Salta,10108838,National University of Salta,[]
2,01yjy8p80,National University of Luján,active,Education,[],[],"[{'label': 'Universidad Nacional de Luján', 'iso639': 'es'}]",Luján,None,Buenos Aires,Argentina,Buenos Aires,168807105,National University of Luján,[]
3,029efta16,Universidad del Salvador,active,Education,[],[USAL],"[{'label': 'Université del salvador', 'iso639': 'fr'}]",Buenos Aires,None,Buenos Aires F.D.,Argentina,Buenos Aires F.D.,11383257,Universidad del Salvador,[]
4,05s004555,Favaloro University,active,Education,[],[],"[{'label': 'Universidad Favaloro', 'iso639': 'es'}]",Buenos Aires,None,Buenos Aires F.D.,Argentina,Buenos Aires F.D.,187398979,Favaloro University,[]


In [116]:
inst_ror['exact_names'] = (inst_ror.apply(lambda x: get_exact_names(x['name'], x.aliases,
                                                                         x.acronyms, x.labels), axis=1))

In [117]:
inst_ror.apply(lambda x: get_exact_names(x['name'], x.aliases,x.acronyms, x.labels), axis=1)

,0
0,"[NationalUniversityoftheLittoral, UniversidadNacionaldelLitoral, UniversiteNationaleduLittoral]"
1,"[NationalUniversityofSalta, UniversidadNacionaldeSalta]"
2,"[NationalUniversityofLujan, UniversidadNacionaldeLujan]"
3,"[UniversidaddelSalvador, Universitedelsalvador]"
4,"[FavaloroUniversity, UniversidadFavaloro]"
...,...
326,"[NationalUniversityofMardelPlata, UNMdP, UniversidadNacionaldeMardelPlata]"
327,"[UniversidadIsalud, ISALUDUniversity]"
328,"[UniversidadNacionaldeCordoba, NationalUniversityofCordoba, UniversidadeNacionaldeCordoba, UniversitaNazionalediCordoba, Universitenationaledecordoba]"
329,"[UniversidadNacionaldeLaPlata, NationalUniversityofLaPlata, Universitenationaledelaplata]"


In [118]:
inst_ror['final_names'] = inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)

In [119]:
inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)

,0
0,"[NationalUniversityoftheLittoral, UniversidadNacionaldelLitoral, UniversiteNationaleduLittoral]"
1,"[UniversidadNacionaldeSalta, NationalUniversityofSalta]"
2,"[UniversidadNacionaldeLujan, NationalUniversityofLujan]"
3,"[UniversidaddelSalvador, Universitedelsalvador]"
4,"[UniversidadFavaloro, FavaloroUniversity]"
...,...
326,"[NationalUniversityofMardelPlata, UNMdP, UniversidadNacionaldeMardelPlata]"
327,"[UniversidadIsalud, ISALUDUniversity]"
328,"[UniversidadeNacionaldeCordoba, NationalUniversityofCordoba, Universitenationaledecordoba, UniversitaNazionalediCordoba, UniversidadNacionaldeCordoba]"
329,"[NationalUniversityofLaPlata, UniversidadNacionaldeLaPlata, Universitenationaledelaplata]"


In [120]:
## lo paso a entero antes de hacerlo indice
inst_ror['affiliation_id']=inst_ror['affiliation_id'].astype(int)

In [121]:
new_affiliation_dict = inst_ror.set_index('affiliation_id')[['display_name','city','state',
                                                             'country','final_names','ror_id','types']] \
.to_dict(orient='index')

In [122]:
new_affiliation_dict

{3121987637: {'display_name': 'National University of the Littoral',
  'city': 'Santa Fe',
  'state': 'Santa Fe',
  'country': 'Argentina',
  'final_names': ['NationalUniversityoftheLittoral',
   'UniversidadNacionaldelLitoral',
   'UniversiteNationaleduLittoral'],
  'ror_id': '00pt8r998',
  'types': 'Education'},
 10108838: {'display_name': 'National University of Salta',
  'city': 'Salta',
  'state': 'Salta',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeSalta', 'NationalUniversityofSalta'],
  'ror_id': '00htwgm11',
  'types': 'Education'},
 168807105: {'display_name': 'National University of Luján',
  'city': 'Luján',
  'state': 'Buenos Aires',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeLujan', 'NationalUniversityofLujan'],
  'ror_id': '01yjy8p80',
  'types': 'Education'},
 11383257: {'display_name': 'Universidad del Salvador',
  'city': 'Buenos Aires',
  'state': 'Buenos Aires F.D.',
  'country': 'Argentina',
  'final_names': ['Universidad

In [123]:
with open(f"{curr_model_artifacts_location}full_affiliation_dict1.pkl", "wb") as f:
    pickle.dump(new_affiliation_dict, f)

In [124]:
new_affiliation_dict

{3121987637: {'display_name': 'National University of the Littoral',
  'city': 'Santa Fe',
  'state': 'Santa Fe',
  'country': 'Argentina',
  'final_names': ['NationalUniversityoftheLittoral',
   'UniversidadNacionaldelLitoral',
   'UniversiteNationaleduLittoral'],
  'ror_id': '00pt8r998',
  'types': 'Education'},
 10108838: {'display_name': 'National University of Salta',
  'city': 'Salta',
  'state': 'Salta',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeSalta', 'NationalUniversityofSalta'],
  'ror_id': '00htwgm11',
  'types': 'Education'},
 168807105: {'display_name': 'National University of Luján',
  'city': 'Luján',
  'state': 'Buenos Aires',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeLujan', 'NationalUniversityofLujan'],
  'ror_id': '01yjy8p80',
  'types': 'Education'},
 11383257: {'display_name': 'Universidad del Salvador',
  'city': 'Buenos Aires',
  'state': 'Buenos Aires F.D.',
  'country': 'Argentina',
  'final_names': ['Universidad

### Updating the city/country file

This file is used to check the affiliation string to make sure it doesn't exactly match up with a city/region/country combo with no additional information.

In [125]:
city_region_country = inst_ror.drop_duplicates(subset=['city','country']).copy()
city_region_country.shape

(59, 17)

In [126]:
new_city_country_list = list(set([f"{i}{j}" for i,j in zip(city_region_country['city'].tolist(),
                                   city_region_country['country'].tolist())] +
         [f"{i}{j}{k}"for i,j,k in zip(city_region_country['city'].tolist(),
                                             city_region_country['state'].tolist(),
                                             city_region_country['country'].tolist()) if j ] +
         [f"{i}{j}" for i,j in zip(city_region_country['state'].tolist(),
                                   city_region_country['country'].tolist()) if i] +
         [f"{i}" for i in city_region_country['country'].tolist()] +
         [f"{i}" for i in city_region_country['state'].tolist() if i]))

new_city_country_list = list(set([string_match_clean(x) for x in new_city_country_list]))

In [127]:
len(new_city_country_list)

149

In [128]:
with open(f"{curr_model_artifacts_location}city_country_list1.pkl", "wb") as f:
    pickle.dump(new_city_country_list, f)

### Flat country file is up to date

Flat country file is needed to search for country in the string for the model.

In [129]:
len(list(set(countries_list_flat)))

396

In [130]:
all_countries = []
for i in countries_dict.values():
    all_countries += i

In [131]:
len(list(set(all_countries)))

396

In [132]:
with open(f"{curr_model_artifacts_location}countries_list_flat1.pkl", "wb") as f:
    pickle.dump(list(set(all_countries)), f)

### Departments list update

Takes the old department list and updates it with additional department names.

In [133]:
file_path = os.path.join(base_path, 'V2/001_Exploration/ror_string_beginnings/Education_dept')
with open(file_path, 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [134]:
departments_list = ['Psychology','Nephrology','Other departments','Other Departments','Nursing & Midwifery',
                    'Literature and Creative Writing','Neuroscience','Engineering','Computer Science',
                    'Chemistry','Biology','Medicine']

In [135]:
new_departments_list = list(set(departments_list + education_dept_begs))

In [136]:
with open(f"{curr_model_artifacts_location}departments_list1.pkl", "wb") as f:
    pickle.dump(new_departments_list, f)

### Make affiliation IDs integers

In [137]:
with open(f"{curr_model_artifacts_location}affiliation_vocab.pkl", "rb") as f:
    affiliation_vocab_basic = pickle.load(f)

print(affiliation_vocab_basic)
print("Loaded affiliation vocab basic")


new_affiliation_vocab_basic = {int(i):int(j) for j,i in affiliation_vocab_basic.items()}
print(new_affiliation_vocab_basic)
print("New vocab basic")


Output hidden; open in https://colab.research.google.com to view.

In [138]:
with open(f"{curr_model_artifacts_location}new_affiliation_vocab1.pkl", "wb") as f:
    pickle.dump(new_affiliation_vocab_basic, f)

In [139]:
print(new_affiliation_vocab_basic)

{0: 4210101023, 1: 4210152343, 2: 4210151182, 3: 4210116154, 4: 4210147808, 5: 4210125984, 6: 4210139853, 7: 4210154781, 8: 2801859090, 9: 4210119632, 10: 4210127757, 11: 4210111023, 12: 4210139907, 13: 4210109317, 14: 4210152680, 15: 4210118184, 16: 4210090315, 17: 4210163453, 18: 4210138920, 19: 4210099283, 20: 4210096141, 21: 2802304995, 22: 4210147683, 23: 4210166596, 24: 4210092085, 25: 4210142358, 26: 4210111346, 27: 4210127876, 28: 4210116813, 29: 4210098210, 30: 4210145978, 31: 4210133595, 32: 2801215448, 33: 4210121081, 34: 4210157222, 35: 4210136239, 36: 4210108110, 37: 4210093350, 38: 4210092323, 39: 4210110906, 40: 4210132792, 41: 4210139592, 42: 4210113451, 43: 4210147758, 44: 4210090661, 45: 4210166813, 46: 4210118837, 47: 4210096969, 48: 4210130407, 49: 2799426268, 50: 4210092749, 51: 4210089116, 52: 4210139426, 53: 4210111910, 54: 4210128881, 55: 4210154507, 56: 4210160357, 57: 4210098666, 58: 4210167384, 59: 5429594, 60: 4210106233, 61: 4210143092, 62: 4210157111, 63: 

In [143]:
!pip install colab-convert
!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.py

  Preparing metadata (setup.py) ... done
  Created wheel for colab-convert: filename=colab_convert-2.0.5-py3-none-any.whl size=19207 sha256=2f3b1ab7181d077d44fa5dc93f27a163ef31d89e4566cd1e482c2307b16cd7df
  Stored in directory: /root/.cache/pip/wheels/a7/9f/0a/70f4be5eeba4a3fca9d7bcf68d5a4e97edb9f22be449cc1e8c
Successfully built colab-convert

---------------------------
----------[START]----------
---------------------------
[OK]   en_US Detected as a supported Language
[OK]   convert magic commands
[OK]   commenting out unsupported magic commands
[OK]   keeping new imports made by converter
[NOT]  showing outputs from converter

convert /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.ipynb to /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.py
finished
conversion took 1.019169 seconds

log file created:
/content/cc-outputs.log
